<a href="https://colab.research.google.com/github/tmgrgg/nlp-mafiascum/blob/master/BERT_Mafiascum_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# 6 - Transformers for Sentiment Analysis

In this notebook we will be using the transformer model, first introduced in [this](https://arxiv.org/abs/1706.03762) paper. Specifically, we will be using the BERT (Bidirectional Encoder Representations from Transformers) model from [this](https://arxiv.org/abs/1810.04805) paper. 

Transformer models are considerably larger than anything else covered in these tutorials. As such we are going to use the [transformers library](https://github.com/huggingface/transformers) to get pre-trained transformers and use them as our embedding layers. We will freeze (not train) the transformer and only train the remainder of the model which learns from the representations produced by the transformer. In this case we will be using a multi-layer bi-directional GRU, however any model can learn from these representations.

In [2]:
import pandas as pd
import torch
from torchtext import data
from torchtext import datasets
import random
import numpy as np

SEED = 49
torch.backends.cudnn.deterministic = True
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)

DOCS_PATH = 'drive/My Drive/MAFIA_DATA/24h_of_deception_basic.pkl'

docs = pd.read_pickle(DOCS_PATH)
docs = docs.sample(frac=1)

print(len(docs))

# tokeniztion is done by BERT
docs['content'] = docs['words'].apply(lambda x: ' '.join(x))

TEST_RATIO = 0.5
VALID_RATIO = 0.3

# split at user level to stop leakage (plus, we want to be agnostic to author identity!)
users = docs.author.unique()
np.random.shuffle(users)

train_users = users[int(len(users)*TEST_RATIO):]
test_users = users[:int(len(users)*TEST_RATIO)]
valid_users = train_users[:int(len(train_users)*VALID_RATIO)]
train_users = train_users[int(len(train_users)*VALID_RATIO):]

train_docs = docs[np.isin(docs['author'].values, train_users)]
valid_docs = docs[np.isin(docs['author'].values, valid_users)]
test_docs = docs[np.isin(docs['author'].values, test_users)]

print(docs.iloc[0]['author'] == 'Egg')
print(docs.iloc[1]['author'] == 'Wake88')
print(train_docs.iloc[0]['author'] == 'Egg')
print(train_docs.iloc[1]['author'] == 'Vendagoat')
print(valid_docs.iloc[0]['author'] == 'Wake88')
print(valid_docs.iloc[1]['author'] == 'OhGodMyLife')
print(test_docs.iloc[0]['author'] == 'Bellaphant')
print(test_docs.iloc[1]['author'] == 'Fishbulb')

112523
False
False
False
False
False
False
False
False


## Preparing Data

First, as always, let's set the random seeds for deterministic results.

The transformer has already been trained with a specific vocabulary, which means we need to train with the exact same vocabulary and also tokenize our data in the same way that the transformer did when it was initially trained.

Luckily, the transformers library has tokenizers for each of the transformer models provided. In this case we are using the BERT model which ignores casing (i.e. will lower case every word). We get this by loading the pre-trained `bert-base-uncased` tokenizer.

In [3]:
!pip install transformers

     |████████████████████████████████| 542kB 2.8MB/s 
     |████████████████████████████████| 870kB 43.9MB/s 
     |████████████████████████████████| 1.0MB 33.7MB/s 
     |████████████████████████████████| 3.7MB 31.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=4188c9e81402d41770772a14e09513fcd3abcc90495da8f0f2ed50b42670761f
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [4]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

The `tokenizer` has a `vocab` attribute which contains the actual vocabulary we will be using. We can check how many tokens are in it by checking its length.

In [5]:
len(tokenizer.vocab)

30522

Using the tokenizer is as simple as calling `tokenizer.tokenize` on a string. This will tokenize and lower case the data in a way that is consistent with the pre-trained transformer model.

In [6]:
tokens = tokenizer.tokenize('Hello WORLD how ARE yoU?')

print(tokens)

['hello', 'world', 'how', 'are', 'you', '?']


We can numericalize tokens using our vocabulary using `tokenizer.convert_tokens_to_ids`.

In [7]:
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

[7592, 2088, 2129, 2024, 2017, 1029]


The transformer was also trained with special tokens to mark the beginning and end of the sentence, detailed [here](https://huggingface.co/transformers/model_doc/bert.html#transformers.BertModel). As well as a standard padding and unknown token. We can also get these from the tokenizer.

**Note**: the tokenizer does have a beginning of sequence and end of sequence attributes (`bos_token` and `eos_token`) but these are not set and should not be used for this transformer.

In [8]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


We can get the indexes of the special tokens by converting them using the vocabulary...

In [9]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


...or by explicitly getting them from the tokenizer.

In [10]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


Another thing we need to handle is that the model was trained on sequences with a defined maximum length - it does not know how to handle sequences longer than it has been trained on. We can get the maximum length of these input sizes by checking the `max_model_input_sizes` for the version of the transformer we want to use. In this case, it is 512 tokens.

In [11]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

512


Previously we have used the `spaCy` tokenizer to tokenize our examples. However we now need to define a function that we will pass to our `TEXT` field that will handle all the tokenization for us. It will also cut down the number of tokens to a maximum length. Note that our maximum length is 2 less than the actual maximum length. This is because we need to append two tokens to each sequence, one to the start and one to the end.

In [0]:
def tokenize_and_cut(sentence):
    #print('tokenizing')
    tokens = tokenizer.tokenize(sentence) 
    #print(tokens)
    tokens = tokens[:max_input_length-2]
    #print(len(tokens))
    return tokens

Now we define our fields. The transformer expects the batch dimension to be first, so we set `batch_first = True`. As we already have the vocabulary for our text, provided by the transformer we set `use_vocab = False` to tell torchtext that we'll be handling the vocabulary side of things. We pass our `tokenize_and_cut` function as the tokenizer. The `preprocessing` argument is a function that takes in the example after it has been tokenized, this is where we will convert the tokens to their indexes. Finally, we define the special tokens - making note that we are defining them to be their index value and not their string value, i.e. `100` instead of `[UNK]` This is because the sequences will already be converted into indexes.

We define the label field as before.

In [0]:
from torchtext import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  include_lengths=True,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [0]:
from torchtext import data

class DataFrameDataset(data.Dataset):

    def __init__(self, df, text_field, label_field, **kwargs):
        fields = [('text', text_field), ('label', label_field), ('index', Field(sequential=False, use_vocab=False))]
        examples = []
        for i, row in df.iterrows():
            label = row.scum 
            text = row.content
            examples.append(data.Example.fromlist([text, label, i], fields))

        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex):
        return len(ex.text)

from torchtext.data import Field, LabelField

# define Fields
#TEXT = Field(sequential=True, include_lengths=True, batch_first=True)
#LABEL = LabelField(dtype=torch.float)
#LABEL = LabelField(dtype=torch.long)

# define Dataset
#train_data = 
#dataset = DataFrameDataset(docs, TEXT, LABEL)
train_data = DataFrameDataset(train_docs, TEXT, LABEL)
valid_data = DataFrameDataset(valid_docs, TEXT, LABEL)
test_data = DataFrameDataset(test_docs, TEXT, LABEL)

#train_data, test_data = dataset.split(split_ratio=0.8, random_state=random.seed(SEED))
#train_data, valid_data = train_data.split(split_ratio = 0.8, random_state= random.seed(SEED))

In [15]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 39847
Number of validation examples: 16470
Number of testing examples: 56206


We can check an example and ensure that the text has already been numericalized.

In [16]:
print(vars(train_data.examples[6]))

{'text': [3789, 21461, 7301, 15315, 6633, 2057, 2310, 2288, 1037, 8040, 2819, 25154, 2182, 12455, 3733, 2769, 3789, 2417, 3597, 7677, 2618, 1045, 2066, 2108, 8075, 9906, 2036, 2108, 2006, 1037, 4440, 2000, 3146, 2005, 1037, 10355, 4680, 1998, 3225, 1037, 2047, 3105, 2003, 13939, 2635, 2039, 2026, 2051, 2017, 2323, 3046, 2000, 2067, 2023, 2039, 2007, 3563, 2015, 14686, 22073, 4964, 8239, 16614, 14525, 2024, 4569, 4521, 1996, 2659, 5689, 5909, 4364, 2065, 2017, 2371, 2066, 2045, 2001, 1037, 3768, 1997, 4180, 2220, 2006, 1999, 1996, 2208, 4276, 10537, 2339, 2052, 2017, 2215, 2000, 5245, 1996, 2208, 1999, 2023, 3257, 2023, 5683, 2066, 1037, 6881, 2785, 1997, 5600, 19777, 2005, 3343, 6830, 4364, 1996, 3768, 1997, 4180, 1999, 2023, 2208, 2003, 2919, 1045, 1049, 2074, 6069, 2191, 10021, 8466, 2012, 1996, 2711, 1045, 1049, 6830, 1045, 1049, 2893, 1037, 3492, 5024, 3110, 2008, 1996, 5450, 1999, 2029, 22110, 2003, 8455, 2023, 2208, 2003, 2200, 21249, 1045, 2131, 1996, 8605, 2008, 22110, 2003, 26

We can use the `convert_ids_to_tokens` to transform these indexes back into readable tokens.

In [17]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['text'])

print(tokens)

['vote', 'ff', '##ery', 'sk', '##em', 'we', 've', 'got', 'a', 'sc', '##um', '##claim', 'here', 'folks', 'easy', 'money', 'vote', 'red', '##co', '##yo', '##te', 'i', 'like', 'being', 'mysterious', 'lately', 'also', 'being', 'on', 'a', 'trip', 'to', 'texas', 'for', 'a', 'gaming', 'convention', 'and', 'starting', 'a', 'new', 'job', 'is', 'strangely', 'taking', 'up', 'my', 'time', 'you', 'should', 'try', 'to', 'back', 'this', 'up', 'with', 'specific', '##s', 'quote', 'tags', 'fixed', 'fucking', 'quotes', 'wagons', 'are', 'fun', 'eat', 'the', 'low', 'hanging', 'fruit', 'guys', 'if', 'you', 'felt', 'like', 'there', 'was', 'a', 'lack', 'of', 'content', 'early', 'on', 'in', 'the', 'game', 'worth', 'discussing', 'why', 'would', 'you', 'want', 'to', 'push', 'the', 'game', 'in', 'this', 'direction', 'this', 'feels', 'like', 'a', 'weird', 'kind', 'of', 'defensive', 'justification', 'for', 'policy', 'voting', 'guys', 'the', 'lack', 'of', 'content', 'in', 'this', 'game', 'is', 'bad', 'i', 'm', 'just

Although we've handled the vocabulary for the text, we still need to build the vocabulary for the labels.

In [0]:
LABEL.build_vocab(train_data)

In [19]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f4df19dd620>, {False: 0, True: 1})


As before, we create the iterators. Ideally we want to use the largest batch size that we can as I've found this gives the best results for transformers.

In [0]:
BATCH_SIZE = 256

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if False:
  train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
      (train_data, valid_data, test_data), 
      batch_size = BATCH_SIZE, 
      device = device)

train_iterator = data.BucketIterator(train_data, train=True, sort=True, batch_size = BATCH_SIZE, device = device)
valid_iterator = data.BucketIterator(valid_data, train=False, sort=True, batch_size = BATCH_SIZE, device = device)

## Build the Model

Next, we'll load the pre-trained model, making sure to load the same model as we did for the tokenizer.

In [21]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

Next, we'll define our actual model. 

Instead of using an embedding layer to get embeddings for our text, we'll be using the pre-trained transformer model. These embeddings will then be fed into a GRU to produce a prediction for the sentiment of the input sentence. We get the embedding dimension size (called the `hidden_size`) from the transformer via its config attribute. The rest of the initialization is standard.

Within the forward pass, we wrap the transformer in a `no_grad` to ensure no gradients are calculated over this part of the model. The transformer actually returns the embeddings for the whole sequence as well as a *pooled* output. The [documentation](https://huggingface.co/transformers/model_doc/bert.html#transformers.BertModel) states that the pooled output is "usually not a good summary of the semantic content of the input, you’re often better with averaging or pooling the sequence of hidden-states for the whole input sequence", hence we will not be using it. The rest of the forward pass is the standard implementation of a recurrent model, where we take the hidden state over the final time-step, and pass it through a linear layer to get our predictions.

In [0]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
        #print(text.shape)
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

Next, we create an instance of our model using standard hyperparameters.

We can check how many parameters the model has. Our standard models have under 5M, but this one has 112M! Luckily, 110M of these parameters are from the transformer and we will not be training those.

We can double check the names of the trainable parameters, ensuring they make sense. As we can see, they are all the parameters of the GRU (`rnn`) and the linear layer (`out`).

## Train the Model

As is standard, we define our optimizer and criterion (loss function).

Next, we'll define functions for: calculating accuracy, performing a training epoch, performing an evaluation epoch and calculating how long a training/evaluation epoch takes.

In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
from sklearn.metrics import roc_auc_score, average_precision_score

def train(model, iterator, optimizer, criterion, weight_by_length=False):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    all_predictions = []
    all_y_true = []
    
    for batch in iterator:       
        optimizer.zero_grad()
        
        text, text_lengths = batch.text

        predictions = model(text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)

        if weight_by_length:
          sample_weight = torch.log(text_lengths.float())
          #print(sample_weight)
          loss = sample_weight * loss

        loss = loss.mean()
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()

        all_predictions.append(predictions.cpu().detach().numpy())
        all_y_true.append(batch.label.cpu().detach().numpy())

    
    all_y_true = np.hstack(all_y_true)
    all_predictions = np.hstack(all_predictions)
    auc_score = roc_auc_score(all_y_true, all_predictions)   
    ap_score = average_precision_score(all_y_true, all_predictions) 
    
    return epoch_loss / len(iterator), epoch_acc / len(iterator), auc_score, ap_score

In [0]:
def evaluate(model, iterator, criterion, weight_by_length=False):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        all_predictions = []
        all_y_true = []
    
        for batch in iterator:
            text, text_lengths = batch.text
            predictions = model(text).squeeze(1)
            
            loss = criterion(predictions, batch.label)

            if weight_by_length:
              sample_weight = torch.log(text_lengths.float())
              #print(sample_weight)
              loss = sample_weight * loss

            loss = loss.mean()
                
            acc = binary_accuracy(predictions, batch.label)

            all_predictions.append(predictions.cpu().numpy())
            all_y_true.append(batch.label.cpu().numpy())

            epoch_loss += loss.item()
            epoch_acc += acc.item()
    
    all_y_true = np.hstack(all_y_true)
    all_predictions = np.hstack(all_predictions)
    auc_score = roc_auc_score(all_y_true, all_predictions)   
    ap_score = average_precision_score(all_y_true, all_predictions)  
    return epoch_loss / len(iterator), epoch_acc / len(iterator), auc_score, ap_score

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
from collections import OrderedDict

class StatsTracker():
  def __init__(self, plot_freq=5):
    self.counter = 0
    self.plot_freq = plot_freq
    self._metrics = OrderedDict({})

  def _make_plot(self):
      fig, ax = plt.subplots(1, len(self._metrics.keys()), squeeze=False, figsize=(18,4))
      for i, metric in enumerate(self._metrics.keys()):
        train_metric = self._metrics[metric]['train']
        valid_metric = self._metrics[metric]['valid']
        ax[0, i].plot(list(range(len(train_metric))), train_metric, c='b', label='Train', marker='.')
        ax[0, i].plot(list(range(len(valid_metric))), valid_metric, c='r', label='Valid', marker='.')
        ax[0, i].set_ylabel(metric)
        ax[0, i].set_xlabel('epochs')
        ax[0, i].legend(loc='upper right')

  def plot(self):
    # will plot the current loss_graph every plot_freq^th call to plot
    self.counter += 1
    print('eyo')
    print(self.counter)
    print(self.counter % self.plot_freq)
    if (self.counter % self.plot_freq == 0):
      print('hey')
      fig = self._make_plot()
      clear_output()
      plt.show()
      #fig.close()

  def save(self, path):
    # save the current loss_graph to the given path
    self._make_plot()
    plt.savefig(path)
    plt.close()

  def log(self, value, metric, setting='train'):
    assert(setting == 'train' or setting == 'valid')
    if metric not in self._metrics.keys():
      self._metrics[metric] = {'train': [], 'valid': []}
    self._metrics[metric][setting].append(value)

Finally, we'll train our model. This takes considerably longer than any of the previous models due to the size of the transformer. Even though we are not training any of the transformer's parameters we still need to pass the data through the model which takes a considerable amount of time on a standard GPU.

In [0]:
HIDDEN_DIM = 100
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [39]:
#criterion = nn.BCEWithLogitsLoss()
pos_weight = len(docs[docs['scum'] == False])/len(docs[docs['scum'] == True])
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.Tensor([pos_weight]), reduction='none')
print('positive weight:', pos_weight)

positive weight: 3.2006570351289803


Place the model and criterion onto the GPU (if available)

In [0]:
model = model.to(device)
criterion = criterion.to(device)

In [0]:
N_EPOCHS = 15
WEIGHT_BY_LENGTH = False
best_valid_loss = float('inf')

tracker = StatsTracker(plot_freq=1)
valid_loss, valid_acc, valid_auc, valid_ap = evaluate(model, valid_iterator, criterion)
tracker.log(valid_loss, 'loss', setting='valid')
tracker.log(valid_acc, 'accuracy', setting='valid')
tracker.log(valid_auc, 'AUC', setting='valid')
tracker.log(valid_ap, 'AP', setting='valid')
for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc, train_auc, train_ap = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc, valid_auc, valid_ap = evaluate(model, valid_iterator, criterion)

    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    tracker.log(valid_loss, 'loss', setting='valid')
    tracker.log(valid_acc, 'accuracy', setting='valid')
    tracker.log(valid_auc, 'AUC', setting='valid')
    tracker.log(valid_ap, 'AP', setting='valid')
    tracker.log(train_loss, 'loss', setting='train')
    tracker.log(train_acc, 'accuracy', setting='train')
    tracker.log(train_auc, 'AUC', setting='train')
    tracker.log(train_ap, 'AP', setting='train')
    tracker.plot()
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Train AUC: {train_auc:.3f} | Train AP: {train_ap:.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% | Val. AUC: {valid_auc:.3f} | Val. AP: {valid_ap:.3f}')


#HIDDEN_DIM = 100
#OUTPUT_DIM = 1
#N_LAYERS = 2
#BIDIRECTIONAL = True
#DROPOUT = 0.25
# write results for future reference ;)
save_path = 'drive/My Drive/model:HIDDEN_DIM[{}]_N_LAYERS[{}]_BIDIRECTIONAL[{}]_DROPOUT[{}]_OPTIM[{}]_WEIGHT_BY_LENGTH[{}]'.format(
    HIDDEN_DIM,
    N_LAYERS,
    BIDIRECTIONAL,
    DROPOUT,
    '_'.join(str(optimizer).split('\n')),
    WEIGHT_BY_LENGTH
)

tracker.save(save_path + '.png')

We'll load up the parameters that gave us the best validation loss and try these on the test set - which gives us our best results so far!

In [0]:
model.load_state_dict(torch.load('tut6-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

## Inference

We'll then use the model to test the sentiment of some sequences. We tokenize the input sequence, trim it down to the maximum length, add the special tokens to either side, convert it to a tensor, add a fake batch dimension and then pass it through our model.

In [0]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [0]:
predict_sentiment(model, tokenizer, "This film is terrible")

In [0]:
predict_sentiment(model, tokenizer, "This film is great")